In [4]:
import sys
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load DMS and store object

In [19]:
sys.path.append('../')

from data_prep import dm_saver, data_manager

dms = dm_saver.DM_Saver.load('../dms')

dml = data_manager.Data_Manager()

dml.dms2dm(dms)
%store dml

Stored 'dml' (Data_Manager)


In [3]:
train_a = dml.data_A.reset_index(drop=True)
train_b = dml.data_B.reset_index(drop=True)
train_c = dml.data_C.reset_index(drop=True)

In [4]:
print(train_a.shape)
print(train_b.shape)
print(train_c.shape)

(34061, 47)
(32819, 47)
(26071, 47)


## Locating constant pvm y-intervals
Dataset B and C has 'holes' in y_train, where pv_measurement is constant. Therefore we don't want to train on this data and can remove the corresponding rows for both X and Y

In [19]:
y_hole_length = 40
from helpers import find_const_interval

In [8]:

print(train_b['pv_measurement'][19608]) # this produced an exception earlier

0.0


In [20]:
# TAKES A WHILE TO RUN
y_train_a_const_idx, ca = find_const_interval(train_a, 'pv_measurement', y_hole_length)
print('y_train_a anomalies:',ca)
y_train_b_const_idx, cb = find_const_interval(train_b, 'pv_measurement', y_hole_length)
print('y_train_b anomalies:',cb)
y_train_c_const_idx, cc = find_const_interval(train_c, 'pv_measurement', y_hole_length)
print('y_train_c anomalies:',cc)


y_train_a anomalies: 1
y_train_b anomalies: 1


KeyError: 21169

### Deleting the rows where pvm is const

In [11]:
date_forecast_a_const = train_a.iloc[y_train_a_const_idx]['date_forecast']
date_forecast_a_const_values = date_forecast_a_const.values
train_a = train_a[~train_a['date_forecast'].isin(date_forecast_a_const_values)]

date_forecast_b_const = train_b.iloc[y_train_b_const_idx]['date_forecast']
date_forecast_b_const_values = date_forecast_b_const.values
train_b = train_b[~train_b['date_forecast'].isin(date_forecast_b_const_values)]

date_forecast_c_const = train_c.iloc[y_train_c_const_idx]['date_forecast']
date_forecast_c_const_values = date_forecast_c_const.values
train_c = train_c[~train_c['date_forecast'].isin(date_forecast_c_const_values)]


In [13]:
print(train_a.shape)
print(train_b.shape)
print(train_c.shape)

(34017, 47)
(26005, 47)
(21171, 47)


In [14]:

dml.data_A = train_a
dml.data_B = train_b
dml.data_C = train_c

,pv_measurement,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,month,hours,sum_rad:W
0,0.00,2019-06-02 22:00:00,7.700,1.22825,0.000000,0.00,0.0,280.299988,0.000,0.000000,...,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.0,1.0,0.066987,0.000000
1,0.00,2019-06-02 23:00:00,7.700,1.22350,0.000000,0.00,0.0,280.299988,0.000,0.000000,...,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.0,1.0,0.017037,0.000000
2,0.00,2019-06-03 00:00:00,7.875,1.21975,0.000000,0.00,0.0,280.649994,0.000,0.000000,...,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.0,1.0,0.000000,0.000000
3,0.00,2019-06-03 01:00:00,8.425,1.21800,208.649994,0.75,0.0,281.674988,0.300,526.775024,...,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.0,1.0,0.017037,0.350000
4,19.36,2019-06-03 02:00:00,8.950,1.21800,32468.150391,23.10,0.0,282.500000,11.975,22068.949219,...,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.0,1.0,0.066987,11.741668


## Storing

In [6]:
%store dml

Stored 'dml' (Data_Manager)


In [18]:
# Save train data as csv (without id and date_foeecast)
XA_csv = dml.data_A.drop(['date_forecast'], axis=1)
XB_csv = dml.data_B.drop(['date_forecast'], axis=1)
XC_csv = dml.data_C.drop(['date_forecast'], axis=1)

XA_csv.to_csv('train_A.csv', index=False)
XB_csv.to_csv('train_B.csv', index=False)
XC_csv.to_csv('train_C.csv', index=False)